In [11]:
import pandas as pd
import numpy as np
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing

from sklearn.utils import shuffle
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import LeaveOneOut

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn import metrics

from scipy.stats import sem, t
from scipy import mean

#Oversampling
# from imblearn.over_sampling import RandomOverSampler
from kmeans_smote import KMeansSMOTE


In [12]:
data = pd.read_csv('prepared Data.csv')
print(data.shape)
data.head()


(922, 33)


,gender,age,SBP,BMI,etiology of CKD,Hb,Alb,Cr,eGFR,CKD_stage,...,eGFR(6M),eGFR(12M),eGFR(18M),eGFR(24M),eGFR(30M),eGFR(36M),eGFR(last visit),average_obs,obsevasion_ duration,fclass
0,2,74,120.0,23.137669,2,12.0,4.0,1.20,89.981926,3,...,26.454698,24.331582,24.682189,21.614854,20.420524,20.420524,18.495328,25.275139,37,0
1,1,57,139.0,28.515625,2,15.9,4.8,0.84,88.330020,2,...,78.287758,71.343858,72.845992,71.908942,71.562914,67.225032,67.225032,72.392152,37,0
2,1,32,154.0,24.582701,4,14.4,4.4,0.87,86.973875,2,...,75.027238,69.595257,68.856399,72.901926,69.749275,69.171408,69.171408,72.694258,36,0
3,1,60,144.0,30.737407,2,14.4,4.7,2.22,86.874201,4,...,26.885061,24.917353,28.581660,29.237135,25.556002,25.183703,25.183703,26.485251,35,0
4,1,49,135.0,23.758726,2,17.0,4.1,1.39,86.782629,3,...,46.978867,45.829455,41.488436,41.801561,38.106104,38.106104,38.106104,43.081581,30,0


In [13]:
col = data.columns

In [14]:
count = Counter(data['fclass'])
count


Counter({0: 611, 1: 170, 5: 51, 2: 42, 6: 48})

# Oversampling

In [8]:


X, y = data.values[:,:-1], data.values[:,-1]
[print('Class {} has {} instances'.format(label, count))
 for label, count in zip(*np.unique(y, return_counts=True))]

kmeans_smote = KMeansSMOTE(
    kmeans_args={
        'n_clusters': 20
    },
    smote_args={
        'k_neighbors': 5
    }
)
X_resampled, y_resampled = kmeans_smote.fit_sample(X, y)

[print('Class {} has {} instances after oversampling'.format(label, count))
 for label, count in zip(*np.unique(y_resampled, return_counts=True))]

Class 0.0 has 611 instances
Class 1.0 has 170 instances
Class 2.0 has 42 instances
Class 5.0 has 51 instances
Class 6.0 has 48 instances


/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 2.0. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 5.0. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 6.0. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_cla

Class 0.0 has 611 instances after oversampling
Class 1.0 has 1934 instances after oversampling
Class 2.0 has 1749 instances after oversampling
Class 5.0 has 1731 instances after oversampling
Class 6.0 has 1737 instances after oversampling


[None, None, None, None, None]

In [9]:
[print('Class {} has {} instances after oversampling'.format(label, count))
 for label, count in zip(*np.unique(y_resampled, return_counts=True))]

Class 0.0 has 611 instances after oversampling
Class 1.0 has 1934 instances after oversampling
Class 2.0 has 1749 instances after oversampling
Class 5.0 has 1731 instances after oversampling
Class 6.0 has 1737 instances after oversampling


[None, None, None, None, None]

In [10]:
[print('Class {} has {} instances'.format(label, count))
 for label, count in zip(*np.unique(y, return_counts=True))]

Class 0.0 has 611 instances
Class 1.0 has 170 instances
Class 2.0 has 42 instances
Class 5.0 has 51 instances
Class 6.0 has 48 instances


[None, None, None, None, None]

In [5]:
# define oversampling strategy
# oversample = RandomOverSampler(sampling_strategy='minority')

## Below code works for binary classification 
# oversample = RandomOverSampler(sampling_strategy=0.5)

In [6]:

# # define oversampling strategy
# oversample = RandomOverSampler(sampling_strategy='minority')

# # fit and apply the transform
# X_over, y_over = oversample.fit_resample(data.values[:,:-1], data.values[:,-1])

# Max = dict(Counter(y_over))[0.0]
# flag = True
# while flag:
#     X_over, y_over = oversample.fit_resample(X_over, y_over)
#     for x in list(dict(Counter(y_over)).values()):
#         if x == Max:
#             flag = False
#         else:
#             flag = True
#             break
        
# # summarize class distribution
# print(Counter(y_over))

In [7]:
# # example of evaluating a decision tree with random oversampling
# from numpy import mean
# from sklearn.datasets import make_classification
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedStratifiedKFold
# from sklearn.tree import DecisionTreeClassifier
# from imblearn.pipeline import Pipeline
# from imblearn.over_sampling import RandomOverSampler
# # define dataset
# X, y = make_classification(n_samples=10000, weights=[0.99], flip_y=0)
# # define pipeline
# steps = [('over', RandomOverSampler()), ('model', DecisionTreeClassifier())]
# pipeline = Pipeline(steps=steps)
# # evaluate pipeline
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# scores = cross_val_score(pipeline, X_over, y_over, scoring='accuracy', cv=cv, n_jobs=-1)
# score = mean(scores)
# print('F1 Score: %.3f' % score)
# scores

In [8]:
# adict = dict()
# adict.update(zip(d.columns,col[:-1]))
# adict

In [9]:
# d = pd.DataFrame(X_over)
# adict = dict()
# adict.update(zip(d.columns,col[:-1]))
# d.rename(columns = adict, inplace =True)
# d['fclass'] = y_over
# d.head()

In [16]:
count = Counter(data['fclass'])
count

Counter({0: 611, 1: 170, 5: 51, 2: 42, 6: 48})

In [17]:
# data = d.copy()

In [18]:
cat0 = shuffle(data[data['fclass']==0])
cat1 = shuffle(data[data['fclass']==1])
cat2 = shuffle(data[data['fclass']==2])
cat5 = shuffle(data[data['fclass']==5])
cat6 = shuffle(data[data['fclass']==6])


In [19]:

lenCat0 = len(cat0.iloc[:,0])
lenCat1 = len(cat1.iloc[:,0])
lenCat2 = len(cat2.iloc[:,0])
lenCat5 = len(cat5.iloc[:,0])
lenCat6 = len(cat6.iloc[:,0])


In [37]:
methodList = []
accRes = []
accConf = []
# sensitivityList = []
# specificityList = []

In [38]:
methods = ['Logistic Regression', 'K Neighbors Classifier', 'Random Forest', 'Gaussian Naive Bayes',\
          'Linear Discriminant Analysis', 'Decision Tree', 'Support Vector Machine', 'Gradient Boosting',\
          'VotingClassifier']

In [39]:
## 10 fold-CV

for method in methods:
    if method == 'Logistic Regression':
        CL = LogisticRegression()
    if method == 'K Neighbors Classifier':
        CL = KNeighborsClassifier(n_neighbors=10)
    if method == 'Random Forest':
        CL = RandomForestClassifier(max_depth=35)
    if method == 'Gaussian Naive Bayes':
        CL = GaussianNB()
    if method == 'Linear Discriminant Analysis':
        CL = LinearDiscriminantAnalysis()
    if method == 'Decision Tree':
        CL = DecisionTreeClassifier()
    if method == 'Support Vector Machine':
        CL = SVC()
    if method == 'Gradient Boosting':
        CL = GradientBoostingClassifier()
    if method == 'VotingClassifier':
        cl1 = LogisticRegression()
        cl2 = KNeighborsClassifier(n_neighbors=10)
        cl3 = RandomForestClassifier(max_depth=35)
        cl4 = GaussianNB()
        cl5 = LinearDiscriminantAnalysis()
        cl6 = DecisionTreeClassifier()
        cl7 = SVC()
        cl8 = GradientBoostingClassifier()
        estimator = [(method[0],cl1), (method[1],cl2), (method[2],cl3), (method[3],cl4),\
                     (method[4],cl5), (method[5],cl6), (method[6],cl7), (method[7],cl8)]
        eclf = VotingClassifier(estimators=estimator,
        voting='soft', weights=[5, 5, 10, 5, 6, 8, 4, 10])

    accList = []
    for j in range(10):

        i = j*.1
        k = (j+1)*.1
        #train
        X_train = cat0.iloc[int(k * lenCat0):,:-1].append(\
                  cat1.iloc[int(k * lenCat1):,:-1].append(\
                  cat2.iloc[int(k * lenCat2):,:-1].append(\
                  cat5.iloc[int(k * lenCat5):,:-1].append(\
                  cat6.iloc[int(k * lenCat6):,:-1].append(\
                  cat0.iloc[:int(i * lenCat0),:-1].append(\
                  cat1.iloc[:int(i * lenCat1),:-1].append(\
                  cat2.iloc[:int(i * lenCat2),:-1].append(\
                  cat5.iloc[:int(i * lenCat5),:-1].append(\
                  cat6.iloc[:int(i * lenCat6),:-1]                                       
                                                         )))))))))

        y_train = cat0.iloc[int(k * lenCat0):,-1].append(\
                  cat1.iloc[int(k * lenCat1):,-1].append(\
                  cat2.iloc[int(k * lenCat2):,-1].append(\
                  cat5.iloc[int(k * lenCat5):,-1].append(\
                  cat6.iloc[int(k * lenCat6):,-1].append(\
                  cat0.iloc[:int(i * lenCat0),-1].append(\
                  cat1.iloc[:int(i * lenCat1),-1].append(\
                  cat2.iloc[:int(i * lenCat2),-1].append(\
                  cat5.iloc[:int(i * lenCat5),-1].append(\
                  cat6.iloc[:int(i * lenCat6),-1]                                         
                                                        )))))))))
#         print(X_train.shape, y_train.shape)
    

        # fit and apply the Oversampling:
        #### Repeatitive oversampling
#         X_over, y_over = oversample.fit_resample(X_train, y_train)

#         Max = dict(Counter(y_over))[0.0]
#         flag = True
#         while flag:
#             X_over, y_over = oversample.fit_resample(X_over, y_over)
#             for x in list(dict(Counter(y_over)).values()):
#                 if x == Max:
#                     flag = False
#                 else:
#                     flag = True
#                     break
#         X_train, y_train = X_over.copy(), y_over.copy()
        # summarize class distribution
#         print(Counter(y_train))

#         print(X_train.shape, y_train.shape)
    

        # fit and apply the Oversampling:
        #### K-means oversampling
        X, y = X_train, y_train
#         [print('Class {} has {} instances'.format(label, count))
#          for label, count in zip(*np.unique(y, return_counts=True))]

        kmeans_smote = KMeansSMOTE(
            kmeans_args={
                'n_clusters': 100
            },
            smote_args={
                'k_neighbors': 10
            }
        )
        X_train, y_train = kmeans_smote.fit_sample(X, y)

#         [print('Class {} has {} instances after oversampling'.format(label, count))
#          for label, count in zip(*np.unique(y_resampled, return_counts=True))]
        # summarize class distribution
#         print(Counter(y_train))

        #test
        X_test = cat0.iloc[int(i * lenCat0):int(k * lenCat0),:-1].append(\
                 cat1.iloc[int(i * lenCat1):int(k * lenCat1),:-1].append(\
                 cat2.iloc[int(i * lenCat2):int(k * lenCat2),:-1].append(\
                 cat5.iloc[int(i * lenCat5):int(k * lenCat5),:-1].append(\
                 cat6.iloc[int(i * lenCat6):int(k * lenCat6),:-1]))))

        y_test = cat0.iloc[int(i * lenCat0):int(k * lenCat0),-1].append(\
                 cat1.iloc[int(i * lenCat1):int(k * lenCat1),-1].append(\
                 cat2.iloc[int(i * lenCat2):int(k * lenCat2),-1].append(\
                 cat5.iloc[int(i * lenCat5):int(k * lenCat5),-1].append(\
                 cat6.iloc[int(i * lenCat6):int(k * lenCat6),-1]))))
#         print(X_test.shape, y_test.shape)
        #preprocessing
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train_transformed = scaler.transform(X_train)
        X_test_transformed = scaler.transform(X_test)
        
        model = CL.fit(X_train, np.array(list(y_train)))
        
        predict = np.array(model.predict(X_test))
        
        cnf_matrix = metrics.confusion_matrix(y_test, predict)
        print(method)
        print(cnf_matrix)
#         #matplotlib inline
#         class_names=[0,1] # name  of classes
#         fig, ax = plt.subplots()
#         tick_marks = np.arange(len(class_names))
#         plt.xticks(tick_marks, class_names)
#         plt.yticks(tick_marks, class_names)

#         # create heatmap
#         sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
#         ax.xaxis.set_label_position("top")
#         plt.tight_layout()
#         plt.title(method+ ' fold of '+str(j), y=1.1)

#         plt.ylabel('Actual label')
#         plt.xlabel('Predicted label')

#         plt.show()


        acc = metrics.accuracy_score(y_test.values, predict)
        accList.append(acc)


    methodList.append(method)
    confidence = 0.95

    naccList = len(accList)
    maccList = np.mean(accList)
    std_erraccList = sem(accList)
    haccList = std_erraccList * t.ppf((1 + confidence) / 2, naccList - 1)




    print("Accuracy:         "+ str(round(maccList,2)) + '  -+' + str(round(haccList,4)))
    accRes.append(round(maccList,2))
    accConf.append(round(haccList,4))


/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 2. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 5. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 6. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_lab

Logistic Regression
[[19  2 13 24  3]
 [ 0  8  2  0  7]
 [ 0  0  3  0  1]
 [ 1  0  1  3  0]
 [ 0  3  0  0  1]]


/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 5. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/home/amin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:124: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
/home/amin/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/amin/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression
[[37  1  8 13  2]
 [ 1  8  1  1  6]
 [ 0  1  1  1  1]
 [ 4  1  0  0  0]
 [ 0  1  2  0  2]]


/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 5. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 6. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/home/amin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:124: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
/home/amin/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22

...

VotingClassifier
[[55  2  2  2  0]
 [ 1 15  0  0  1]
 [ 2  1  1  0  0]
 [ 4  0  0  1  0]
 [ 0  1  2  0  2]]


/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 5. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/home/amin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:124: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


VotingClassifier
[[53  2  2  3  1]
 [ 2 13  0  0  2]
 [ 2  0  1  1  0]
 [ 3  0  1  1  0]
 [ 0  2  0  0  3]]


/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 2. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 5. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/home/amin/.local/lib/python3.6/site-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 6. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_lab

VotingClassifier
[[49  3  3  6  0]
 [ 1 13  0  0  3]
 [ 0  1  3  1  0]
 [ 2  1  1  1  0]
 [ 0  4  0  0  1]]
Accuracy:         0.76  -+0.0398


In [36]:
result = []
result = pd.DataFrame(result)
result['Method'],result['Accuracy'],result['ACC CI'] = methodList,accRes,accConf
result.to_excel('kidney_Classification_Result.xlsx',index=False)
result

,Method,Accuracy,ACC CI
0,Logistic Regression,0.43,0.0510
1,K Neighbors Classifier,0.44,0.0357
2,Random Forest,0.72,0.0328
3,Gaussian Naive Bayes,0.42,0.0373
4,Linear Discriminant Analysis,0.47,0.0274
5,Decision Tree,0.62,0.0597
6,Support Vector Machine,0.70,0.0308
7,Gradient Boosting,0.75,0.0658
8,VotingClassifier,0.76,0.0256


In [32]:
# methodList.append('Logistic Regression')
# confidence = 0.95
# # data = [1, 2, 3, 4, 5]

# naccList = len(accList)
# maccList = np.mean(accList)
# std_erraccList = sem(accList)
# haccList = std_erraccList * t.ppf((1 + confidence) / 2, naccList - 1)




# print("Accuracy:         "+ str(round(maccList,2)) + '  -+' + str(round(haccList,4)))
# accRes.append(round(maccList,2))
# accConf.append(round(haccList,4))

In [ ]:
# loologreg = LogisticRegression()
# loologregModel =loologreg.fit(X_train, np.array(list(y_train)))
# predict = np.array(loologreg.predict(X_test))
# cnf_matrix = metrics.confusion_matrix(y_test, predict)

# print(cnf_matrix)

# acc = metrics.accuracy_score(y_test.values, predict)
# print("Logistic Regression Accuracy by LOOCV", acc)

In [ ]:
# #matplotlib inline
# class_names=[0,1] # name  of classes
# fig, ax = plt.subplots()
# tick_marks = np.arange(len(class_names))
# plt.xticks(tick_marks, class_names)
# plt.yticks(tick_marks, class_names)

# # create heatmap
# sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
# ax.xaxis.set_label_position("top")
# plt.tight_layout()
# plt.title('Logestic Regression Confusion matrix', y=1.1)

# plt.ylabel('Actual label')
# plt.xlabel('Predicted label')

# plt.show()


In [ ]:
# j = 1
# i = j*.1
# k = (j+1)*.1
# #train
# X_train = cat0.iloc[int(k * lenCat0):,:-1].append(\
#           cat1.iloc[int(k * lenCat1):,:-1].append(\
#           cat2.iloc[int(k * lenCat2):,:-1].append(\
#           cat5.iloc[int(k * lenCat5):,:-1].append(\
#           cat6.iloc[int(k * lenCat6):,:-1].append(\
#           cat0.iloc[:int(i * lenCat0),:-1].append(\
#           cat1.iloc[:int(i * lenCat1),:-1].append(\
#           cat2.iloc[:int(i * lenCat2),:-1].append(\
#           cat5.iloc[:int(i * lenCat5),:-1].append(\
#           cat6.iloc[:int(i * lenCat6),:-1]                                       
#                                                  )))))))))

# y_train = cat0.iloc[int(k * lenCat0):,-1].append(\
#           cat1.iloc[int(k * lenCat1):,-1].append(\
#           cat2.iloc[int(k * lenCat2):,-1].append(\
#           cat5.iloc[int(k * lenCat5):,-1].append(\
#           cat6.iloc[int(k * lenCat6):,-1].append(\
#           cat0.iloc[:int(i * lenCat0),-1].append(\
#           cat1.iloc[:int(i * lenCat1),-1].append(\
#           cat2.iloc[:int(i * lenCat2),-1].append(\
#           cat5.iloc[:int(i * lenCat5),-1].append(\
#           cat6.iloc[:int(i * lenCat6),-1]                                         
#                                                 )))))))))
# print(X_train.shape, y_train.shape)

# #test
# X_test = cat0.iloc[int(i * lenCat0):int(k * lenCat0),:-1].append(\
#          cat1.iloc[int(i * lenCat1):int(k * lenCat1),:-1].append(\
#          cat2.iloc[int(i * lenCat2):int(k * lenCat2),:-1].append(\
#          cat5.iloc[int(i * lenCat5):int(k * lenCat5),:-1].append(\
#          cat6.iloc[int(i * lenCat6):int(k * lenCat6),:-1]))))

# y_test = cat0.iloc[int(i * lenCat0):int(k * lenCat0),-1].append(\
#          cat1.iloc[int(i * lenCat1):int(k * lenCat1),-1].append(\
#          cat2.iloc[int(i * lenCat2):int(k * lenCat2),-1].append(\
#          cat5.iloc[int(i * lenCat5):int(k * lenCat5),-1].append(\
#          cat6.iloc[int(i * lenCat6):int(k * lenCat6),-1]))))

In [ ]:
# from sklearn import datasets
# from sklearn.model_selection import cross_val_score
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import VotingClassifier

# # iris = datasets.load_iris()
# # X, y = iris.data[:, 1:3], iris.target

# clf1 = LogisticRegression(random_state=1)
# clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
# clf3 = GaussianNB()

# eclf = VotingClassifier(
#     estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],
#     voting='soft', weights=[3, 3, 1])

# for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
#     scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=10)
#     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))